In [17]:
!pip install kafka-python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [18]:
!pip install avro

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [19]:
# import required libraries
from kafka import KafkaConsumer, KafkaProducer
import avro.schema
import avro.io
import io
import hashlib, json

In [20]:
def serialize(schema, obj):
    bytes_writer = io.BytesIO()
    encoder = avro.io.BinaryEncoder(bytes_writer)
    writer = avro.io.DatumWriter(schema)
    writer.write(obj, encoder)
    return bytes_writer.getvalue()

In [21]:
def deserialize(schema, raw_bytes):
    bytes_reader = io.BytesIO(raw_bytes)
    decoder = avro.io.BinaryDecoder(bytes_reader)
    reader = avro.io.DatumReader(schema)
    return reader.read(decoder)

In [22]:
schema_file = 'transaction.avsc'
txschema = avro.schema.parse(open(schema_file).read())
schema_file = 'submit.avsc'
submitschema = avro.schema.parse(open(schema_file).read())
schema_file = 'result.avsc'
resultschema = avro.schema.parse(open(schema_file).read())

In [23]:
# Connect to kafka broker running in your local host (docker). Change this to your kafka broker if needed
kafka_broker = '35.240.149.229:9092'

In [24]:
producer = KafkaProducer(bootstrap_servers=[kafka_broker])

In [25]:
txschema

In [26]:
txconsumer = KafkaConsumer(
    'transaction',
     bootstrap_servers=[kafka_broker],
     enable_auto_commit=True,
     value_deserializer=lambda x: deserialize(txschema, x))
resultconsumer = KafkaConsumer(
    'result',
     bootstrap_servers=[kafka_broker],
     enable_auto_commit=True,
     value_deserializer=lambda x: deserialize(resultschema, x))

In [27]:
# Read a transaction message from the topic
message = next(txconsumer)

txid = message.value['txid']
payer = message.value['payer']
payee = message.value['payee']
amount = message.value['amount']

print(f'Transaction ID: {txid}\nPayer: {payer}\nPayee: {payee}\nAmount: {amount}')

Transaction ID: TX08762
Payer: A44110
Payee: A09182
Amount: 136


In [28]:
def gen_signature(txid, payer, payee, amount, token):
    o = {'txid': txid, 'payer': payer, 'payee': payee, 'amount': amount, 'token': token}
    return hashlib.md5(json.dumps(o, sort_keys=True).encode('utf-8')).hexdigest()

In [29]:
token = '949bafe92b109890d9e43a7458918e34'
vid = 'V557697' 

signature = gen_signature(txid, payer, payee, amount, token)

submit = {"vid": vid, "txid": txid, "signature": signature}

raw_bytes = serialize(submitschema, submit)
producer.send('submit', value=raw_bytes)


In [30]:
for message in resultconsumer:

    if message.value["vid"] == vid and message.value["txid"] == txid and message.value["code"] == 200:
        print("Verification successful!")
        print("vid:", message.value["vid"])
        print("txid:", message.value["txid"])
        print("Timestamp:", message.value["timestamp"])
        print("Checksum:", message.value["checksum"])
        break
        
resultconsumer.close()


Verification successful!
vid: V557697
txid: TX08762
Timestamp: 1679894354
Checksum: b122a9f6595d503b1a8f92a9a6d1c2a1
